## Writing Scientific Data to Binary Files

Text files are nice for small files, but most scientific data is way too large and complex to be easy to work with as text.  So, we need more flexibility, and for that we use binary files, which allow us to store data however we want.  

But as the superhero CodeMan always says, "With great flexibility come great complexity".  In this section, we'll get a sense of how libraries that help us store scientific data into binary files in a way that gives us both flexibility and simplicity!

Particularly, we'll end up looking at HDF5 ('.h5', '.hdf', '.hdf5) files, which are used as the core file format for a wide variety of popular data formats, and which works as the basic for the NIX files we'll be working with later on to store complex neuroscience data.

### Serializing Numpy arrays directly to a binary File

Can we create our own binary files?  Of course!  Below, we'll make numpy arrays and play around with serializing the numpy 

| **Code** | **Description ** |
| :-- | :-- |
| **`f = open('data.dat', 'wb')`** | Open a new file file, ready for writing binary data |
| **`f = open('data.dat', 'rb')`** | Open a new file file, ready for reading binary data |
| **`f.close()`** | Close the file linked to the file object and release it to the operating system |
| **`x = np.arange(10).astype(np.uint8)`** | Create an array `x` of 10 values (0-9) where each is stored as unsigned, 8-bit integers |
| **`data_bytes = array.tobytes()`** |  Serialize a numpy array `array` to a binary string `data_bytes` |
| **`array = np.frombuffer(data_bytes, dtype=np.uint8)`** | Deserialize a binary string into a numpy array interpreted as being made of unsigned 8-bit integers. |




In [1]:
# %pip install numpy

In [2]:
import numpy as np

**Exercises**

**Example**: Write 5 16-bit integers to the binary file `x1.dat`.

In [3]:
x = np.arange(5).astype(np.int16)
f = open('x1.dat', 'wb')
f.write(x.tobytes())
f.close()

Write 8 8-bit, unsigned integers to the binary file `x2.dat`.

Open up the `x2.dat` file in your text editor (you'll have to insist that it's readable as text, even though it's really not).  What does the data look like, is the data recognizable?  More importantly, how many characters are there in the file, does this seem like the right number of bytes?

Write 8 64-bit, unsigned integers to the binary file `x3.dat`.

Open up the `x3.dat` file in your text editor (you'll have to insist that it's readable as text, even though it's really not).  What does the data look like, is the data recognizable?  More importantly, does it seem like it's got **more** data than the `x2.dat` file?  It should take up 8x as much space, because 64 bits is 8 times more space than 8 bits.

Read in the `x2.dat` file back as a Numpy array, and make sure it was read back correctly  (note: you'll have to tell numpy how the data should be read, most-commonly in the `np.frombuffer()` function). 

Read in the `x3.dat` file back as a Numpy array. 

### Writing Numpy Arrays to NPY and NPZ Files using Numpy

It's challenging to read in data files when you don't know ahead of time how the data is actually stored.  Numpy provides two formats, ".npy" and ".npz" that make reading data 
into Numpy easier by putting the data's format directly into the file itself.  This takes up just a little more space, but it makes the data way easier to work with!

| **Code** | **Description ** |
| :-- | :-- |
| **`np.random.random(size=100).astype(np.float32)`** | Create 100 random float32 values between 0 and 1 |
| **`np.random.randint(1, 10, size=20).astype(np.uint8)`** | Create 20 random 8-bit, unsigned integers between 0 and 10 |
| **`np.save('data.npy', data)`** | Save the `data` array to the `data.npy` file |
| **`np.savez('data.npz', x=x1, y=y1)`** | Save the `x1` and `y1` arrays as `x` and `y` variables in the `data.npz` file |
| **`data = np.load('data.npy')`** | Load the `data` array from the `data.npy` file |

In [4]:
# %pip install numpy

In [5]:
import numpy as np

**Exercises**

**Example**: Create 10 random 16-bit floating numbers and save them to `x1.npy`.

In [6]:
x = np.random.random(size=10).astype(np.float16)
np.save('x1.npy', x)

**Example**: Load the data in `x1.npy`.  Was it saved correctly?

In [7]:
np.load('x1.npy')

array([0.5913, 0.782 , 0.3926, 0.8516, 0.7036, 0.4297, 0.424 , 0.9927,
       0.8354, 0.2898], dtype=float16)

Create 15 random 32-bit integers between 100 and 200 and save them to `x2.npy`.

Read in `x2.npy`.  was it saved correctly?

In a text editor, take a look at how the `x2.npy` file was saved. (Note: you may have to "insist" that the data can be read in the text editor). Can you find where the data's schema is stored in the file?

Save both an array of times (float64) and an array of voltages (uint16) to `ephys.npz`.

In a text editor, take a look at how the `ephys.npz` file was saved. (Note: you may have to "insist" that the data can be read in the text editor). Can you find where the data's schema is stored in the file?

Read in the volts from `ephys.npz`.  Was it saved correctly?

### Writing Mixed-Type, Labelled, Data and Metadata to HDF5 Files using h5py

HDF5 files have a lot more features:
  - They are highly cross-platform and work with a wide variety of tools
  - They can store many different datasets in a single file (or even in multiple linked files)
  - They can store metadata alonside the data
  - They let you store data hierarchically, making a nice dict-like nested organization for your data
  - They can compress your data.
  - They let you work with data that is larger than memory, making it easy to read in only the data that you need.


So many features!  Here, we'll get a basic senses of how they work by using the `h5py` library, which gives us a dict-like, Numpy-native interface to HDF5 files and is used internally by many popular Python frameworks.

| Code | Description |
| :-- | :-- |
| **`f = h5py.File('filename.h5', 'w')`** | Open an h5py file object for writing |
| **`f.close()`** | Closes the h5py file and releases the linked file back to the operating system. |
| **`f.create_dataset('temp', data=x)`** | Write an array called 'temp' with the data in the numy array `x` into the HDF5 file |
| **`f.create_dataset('data/temp', data=x)`** | Write an array called 'temp' in the folder called "data" with the data in the numy array `x` into the HDF5 file |
| **`f.attrs['name'] = 'Session 1'`** | Set an attribute as metadata onto the root group of the HDF5 file -- this works like a normal Python dictionary |
| **`f['x'].attrs['id'] = 'ABC'`** | Set an attribute as metadata onto the 'x' node of the HDF file |



In [8]:
# %pip install h5py numpy

In [9]:
import h5py
import numpy as np

**Exercises**

**Example**: Write an HDF5 File called `temp.h5` with the following schema:
```
root/
└── temp: uint16, 1000 x 1  (temperature measurements over time)
```

In [10]:
temp = np.random.randint(15, 22, size=(1000,1)).astype(np.uint16)


In [11]:
with h5py.File('temp.h5', 'w') as f:
    f.create_dataset('temp', data=temp)
    f['temp'].attrs.update({
        'description': 'Temperature measurements over time'
    })



Write an HDF5 File called `ephys.h5` with the following schema and descriptions:

```
root/
├── time: float32, 1 x 1000 (trial time, in seconds)
├── voltage: int16, 4 x 1000 (voltage measurements for each recording channel)
└──chan_names: S, 4 (channel names for each recording channel)
```

In [12]:
time = np.linspace(0, 3, 1000).astype(np.float32)
voltage = np.random.normal(1, 1, size=(4, 1000)).astype(np.float32)
chan_names = ['CH01', 'CH02', 'CH03', 'CH05']

Write an HDF5 File called `motion_tracking.h5` with the following schema (feel free to skip the descriptions this time):
```
root/
├── session_date: str
├── subject_id: str
├── camera: 
│   ├── black_noise_image: uint16, 640 x 640 x 3 (reference image taken with lights out)
│   ├── image_width: uint16
│   ├── image_height: uint16
│   ├── shutter_speed: uint16
│   └── aperture: float32
│
└── motion_tracking: 
    ├── time: uint32 1 x 3000 (session time, in milliseconds)
    ├── rb_pos: float32  2 x 3 x 3000 (XYZ coordinates of the center of each tracked rigid body)
    ├── rb_rot: float32  2 x 3 x 3000 (XYZ Euler rotations of each tracked rigid body)
    ├── xyz_names: str 1 x 3 (The spatial coordinate names)
    └── rb_names: str 1 x 2 (The name of each rigid body)


```

In [13]:
session_date = '2024-04-22'
subject_id = 'AD11'
camera_black_noise_im = np.random.randint(0, 30, size=(640, 640)).astype(np.uint16)
im_width = 640
im_height = 640
shutter_speed = 800
aperture = 2.8
motion_time = (np.arange(0, 1000, step=1/shutter_speed)[:3000] * 1000).astype(np.uint32)
rb_pos = np.random.random(size=(2, 3, 3000)).astype(np.float32)
rb_rot = np.random.random(size=(2, 3, 3000)).astype(np.float32)
xyz_names = ['X', 'Y', 'Z']
rb_names = ['head', 'tail_base']

### Reading Data from HDF5 Files

| **Code** | **Description** |
| :-- | :-- |
| **`f = h5py.File('file.h5')`** | Opens an h5py file object for reading |
| **`f.close()`** | Closes the h5py file and releases the linked file back to the operating system. |
| **`f.keys()`** | See a list of datasets and groups at the root node |
| **`f.attrs`** | Get the dict-like attributes at the root node |
| **`f.attrs['a']']`** | Get the 'a' attribute at the root node  |
| **`f['x'][:]`** | Read in the 'x' dataset as a numpy array |
| **`f['x'][5:20]`** | Read in a slice of the 'x' dataset as a numpy array |
| **`f['x'].keys()`** | See a list of datasets and groups at the 'x' node |
| **`f['folder']['x']`** | Get tthe 'x' dataset in the 'folder' group |
| **`f['folder/x']`** | (Alternative Syntax) Ge tthe 'x' dataset in the 'folder' group |





**Exercises**

**Example**: From the temperature file, read in only the last 5 temperature measurements as a numpy array.

In [14]:
f = h5py.File('temp.h5')
temp = f['temp'][-5:, :]
f.close()
temp

array([[16],
       [21],
       [21],
       [16],
       [17]], dtype=uint16)

From the ephys file, read in the first 10 voltage measurements as a numpy array.

from the ephys file, get the name of the second recording channel.

From the ephys file, get the description of the voltage dataset

From the motion tracking file, get the all the XYZ positions of the first rigid body during the recording.

From the motion tracking file, get the caemra's shutter speed during the session